# K-최근접 이웃 (K-Nearest Neighbors, KNN)

**학습 목표:**
- 대표적인 **인스턴스 기반 학습(Instance-based learning)**이자, **게으른 학습(Lazy learning)** 알고리즘인 KNN의 동작 원리를 이해합니다.
- 새로운 데이터가 들어왔을 때, 가장 가까운 K개의 훈련 데이터의 레이블을 참조하여 분류하는 과정을 학습합니다.
- KNN 성능에 결정적인 영향을 미치는 하이퍼파라미터 **K(이웃의 수)**의 최적값을 찾는 방법을 배웁니다.
- 거리 기반 알고리즘에서 **특성 스케일링(Feature Scaling)**이 왜 중요한지 직접 확인하고 적용합니다.
- **결정 경계(Decision Boundary)** 시각화를 통해 K값에 따라 모델의 복잡도가 어떻게 변하는지 직관적으로 파악합니다.

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from matplotlib.colors import ListedColormap

### (1) 데이터 준비 및 탐색 (EDA)
붓꽃(Iris) 데이터셋을 사용합니다. 4개의 특성(꽃받침/꽃잎의 길이/너비)을 사용하여 3가지 품종을 분류하는 문제입니다. KNN의 동작을 시각적으로 이해하기에 적합한 데이터셋입니다.

In [ ]:
iris = load_iris()
X = iris.data
y = iris.target

# 데이터프레임으로 변환하여 확인
iris_df = pd.DataFrame(X, columns=iris.feature_names)
iris_df['species'] = y

print(iris_df.head())

# Pairplot으로 특성 간 관계 및 클래스 분포 확인
sns.pairplot(iris_df, hue='species', palette='bright')
plt.show()

### (2) 특성 스케일링의 중요성
KNN은 데이터 포인트 간의 거리를 측정하여 예측을 수행합니다. 만약 특성들의 값 범위(scale)가 크게 다르면, 값의 범위가 큰 특정 특성이 거리에 더 큰 영향을 미치게 되어 모델이 왜곡될 수 있습니다. 따라서 모든 특성이 예측에 동등하게 기여하도록 스케일을 맞춰주는 것이 매우 중요합니다.

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42, stratify=y)

# 스케일링 전 모델 학습
knn_unscaled = KNeighborsClassifier(n_neighbors=5)
knn_unscaled.fit(X_train, y_train)
pred_unscaled = knn_unscaled.predict(X_test)
acc_unscaled = accuracy_score(y_test, pred_unscaled)
print(f"Accuracy (before scaling): {acc_unscaled:.4f}")

# StandardScaler로 데이터 스케일링
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)

# 스케일링 후 모델 학습
knn_scaled = KNeighborsClassifier(n_neighbors=5)
knn_scaled.fit(X_train_scaled, y_train)
pred_scaled = knn_scaled.predict(X_test_scaled)
acc_scaled = accuracy_score(y_test, pred_scaled)
print(f"Accuracy (after scaling):  {acc_scaled:.4f}")
print("\n(주: Iris 데이터는 특성 간 스케일 차이가 크지 않아 성능 향상이 미미할 수 있지만, 일반적인 데이터에서는 스케일링이 성능에 큰 영향을 미칩니다.)")

### (3) 최적의 K값 찾기
K값은 KNN 모델의 복잡도를 결정하는 중요한 하이퍼파라미터입니다.
- **작은 K**: 모델이 매우 민감하고 복잡해져, 훈련 데이터의 노이즈에 과적합(overfitting)될 수 있습니다.
- **큰 K**: 모델이 너무 단순해져, 데이터의 미세한 패턴을 놓치는 과소적합(underfitting)이 발생할 수 있습니다.

다양한 K값에 대한 정확도를 측정하여 최적의 지점을 찾습니다. 이를 **엘보우 방법(Elbow Method)**이라고도 합니다.

In [ ]:
neighbors = range(1, 26)
train_accuracy = []
test_accuracy = []

for k in neighbors:
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    
    # 훈련 및 테스트 정확도 기록
    train_accuracy.append(accuracy_score(y_train, knn.predict(X_train_scaled)))
    test_accuracy.append(accuracy_score(y_test, knn.predict(X_test_scaled)))

# K값에 따른 정확도 변화 시각화
plt.figure(figsize=(10, 6))
plt.plot(neighbors, train_accuracy, label='Training Accuracy')
plt.plot(neighbors, test_accuracy, label='Testing Accuracy')
plt.title('KNN: Number of Neighbors vs. Accuracy')
plt.xlabel('Number of Neighbors (K)')
plt.ylabel('Accuracy')
plt.xticks(neighbors, rotation=45)
plt.legend()
plt.grid(True)
plt.show()

best_k = test_accuracy.index(max(test_accuracy)) + 1
print(f"최적의 K값은 {best_k}으로 보입니다.")

### (4) 최종 모델 훈련 및 평가
찾아낸 최적의 K값(K=3)으로 최종 모델을 학습하고, 혼동 행렬(Confusion Matrix)과 분류 리포트(Classification Report)를 통해 성능을 상세히 평가합니다.

In [ ]:
final_knn = KNeighborsClassifier(n_neighbors=best_k)
final_knn.fit(X_train_scaled, y_train)
y_pred_final = final_knn.predict(X_test_scaled)

# 혼동 행렬 시각화
cm = confusion_matrix(y_test, y_pred_final)
plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', 
            xticklabels=iris.target_names, yticklabels=iris.target_names)
plt.title(f'Confusion Matrix for KNN (K={best_k})')
plt.xlabel('Predicted Label')
plt.ylabel('True Label')
plt.show()

# 분류 리포트 출력
print(f"\n--- Classification Report (K={best_k}) ---")
print(classification_report(y_test, y_pred_final, target_names=iris.target_names))

### (5) 결정 경계 시각화
모델이 각 클래스를 어떻게 구분하는지 시각적으로 확인하기 위해 결정 경계를 그려봅니다. 여기서는 2개의 특성('petal length', 'petal width')만 사용하여 2차원 평면에 시각화합니다.

In [ ]:
def plot_decision_boundaries(X, y, model, title):
    h = .02  # step size in the mesh
    # Create color maps
    cmap_light = ListedColormap(['#FFAAAA', '#AAFFAA', '#AAAAFF'])
    cmap_bold = ListedColormap(['#FF0000', '#00FF00', '#0000FF'])
    
    # 2개의 특성만 사용
    X_2d = X[:, 2:4] 
    scaler_2d = StandardScaler().fit(X_2d)
    X_2d_scaled = scaler_2d.transform(X_2d)
    
    model.fit(X_2d_scaled, y)
    
    # Plot the decision boundary.
    x_min, x_max = X_2d_scaled[:, 0].min() - 1, X_2d_scaled[:, 0].max() + 1
    y_min, y_max = X_2d_scaled[:, 1].min() - 1, X_2d_scaled[:, 1].max() + 1
    xx, yy = np.meshgrid(np.arange(x_min, x_max, h),
                         np.arange(y_min, y_max, h))
    Z = model.predict(np.c_[xx.ravel(), yy.ravel()])

    # Put the result into a color plot
    Z = Z.reshape(xx.shape)
    plt.figure(figsize=(8, 6))
    plt.pcolormesh(xx, yy, Z, cmap=cmap_light, shading='auto')

    # Plot also the training points
    plt.scatter(X_2d_scaled[:, 0], X_2d_scaled[:, 1], c=y, cmap=cmap_bold, edgecolor='k', s=20)
    plt.xlim(xx.min(), xx.max())
    plt.ylim(yy.min(), yy.max())
    plt.title(title)
    plt.xlabel('Petal Length (scaled)')
    plt.ylabel('Petal Width (scaled)')
    plt.show()

# K=1 (과적합) vs 최적의 K 결정 경계 비교
plot_decision_boundaries(X_train, y_train, KNeighborsClassifier(n_neighbors=1), "KNN Decision Boundary (K=1, Overfitting)")
plot_decision_boundaries(X_train, y_train, KNeighborsClassifier(n_neighbors=best_k), f"KNN Decision Boundary (K={best_k}, Optimal)")